In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from scipy.stats import spearmanr, pearsonr
from joblib import Parallel, delayed
from sklearn.svm import SVR

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
rna = pd.read_csv('results/rna_data.csv', index_col = 0)
relevant_genes = np.array(pd.read_csv('./results/relevant_genes.csv'))
rna = rna.iloc[:,np.isin(rna.columns, relevant_genes)]
            
rna['cell_line'] = rna.index
rna

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,OR8K3,ADORA3,SCO2,TBCE,CCDC39,ARHGAP11B,POLR2J3,NPBWR1,CDR1,cell_line
ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,4.739848,...,0.0,0.000000,3.742006,5.742545,1.560715,1.641546,5.612352,0.275007,0.000000,ACH-001113
ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,3.481557,...,0.0,0.028569,2.946731,5.713971,1.327687,2.247928,4.733354,0.014355,0.000000,ACH-001289
ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,3.273516,...,0.0,0.137504,4.011675,4.690417,0.275007,2.056584,4.807355,0.084064,0.042644,ACH-001339
ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,3.956986,...,0.0,0.000000,3.664483,4.131754,0.555816,1.664483,3.702658,0.028569,0.000000,ACH-001538
ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,4.568032,...,0.0,0.000000,4.275007,4.632268,0.333424,3.173127,4.785551,0.464668,0.000000,ACH-000242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000285,0.056584,0.000000,6.604368,3.266037,4.973152,0.411426,0.097611,0.704872,4.829850,5.178715,...,0.0,0.056584,3.575312,5.513175,0.695994,3.667892,6.325530,0.000000,0.042644,ACH-000285
ACH-002669,3.111031,0.000000,7.031329,1.541019,3.664483,0.014355,3.624101,6.805421,4.472488,4.397118,...,0.0,0.000000,1.875780,3.264536,0.150560,1.687061,5.073392,0.014355,0.000000,ACH-002669
ACH-001858,4.390943,0.000000,7.013239,1.887525,3.252476,0.028569,3.286881,6.902194,5.410748,3.401903,...,0.0,0.028569,3.257011,4.884598,0.310340,1.867896,4.486714,1.464668,0.526069,ACH-001858
ACH-001997,5.057450,0.000000,7.815191,2.538538,3.893362,0.028569,4.079805,6.971659,4.469886,3.463361,...,0.0,0.000000,4.004501,4.004501,0.443607,1.263034,4.595146,0.000000,0.000000,ACH-001997


In [3]:
fold = 4

In [4]:


train_test_names = pd.read_csv('./results/train_test_splits/train_test_names' +str(fold) +'.csv', index_col=0)
train_names = train_test_names[train_test_names['train_test']=='train']['cell_line']
test_names = train_test_names[train_test_names['train_test']=='test']['cell_line']

train_rna = rna[np.isin(rna.index, train_names)]
test_rna = rna[np.isin(rna.index, test_names)]

In [5]:
experiments = pd.read_csv('./results/prediction_targets.csv', index_col = 0)
train_experiments = experiments[np.isin(experiments['cell_line'], train_names)]
test_experiments = experiments[np.isin(experiments['cell_line'], test_names)]
train_experiments
print(test_experiments.shape,train_experiments.shape)

(124038, 6) (422608, 6)


In [6]:
#data_labels_train = train_experiments.merge(train_rna, how='inner')
#data_labels_test = test_experiments.merge(test_rna, how='inner')

#data_labels_test
drugs = experiments['DRUG'].drop_duplicates()
drugs

0          CYTARABINE
398        EPINASTINE
801       FLOXURIDINE
1258       VALRUBICIN
1716        ADAPALENE
             ...     
537605    PARDOPRUNOX
538118      RUBITECAN
538658         XL-647
539493      VERUBULIN
539958         VE-822
Name: DRUG, Length: 1135, dtype: object

In [7]:
experiments

,cell_line,DRUG,auc,auc_per_drug,means,stds
0,ACH-000879,CYTARABINE,1.677789,2.667240,1.266269,0.154287
1,ACH-000320,CYTARABINE,1.240300,-0.168314,1.266269,0.154287
2,ACH-001145,CYTARABINE,1.472333,1.335587,1.266269,0.154287
3,ACH-000873,CYTARABINE,1.207160,-0.383109,1.266269,0.154287
4,ACH-000855,CYTARABINE,1.229332,-0.239405,1.266269,0.154287
...,...,...,...,...,...,...
546641,ACH-001321,CINACALCET,0.987842,0.305002,0.947098,0.133586
546642,ACH-001321,INC-280,1.000000,-0.424494,1.098950,0.233102
546643,ACH-001321,DOVITINIB,0.855352,-0.548900,0.913856,0.106585
546644,ACH-001321,LINSITINIB,0.890923,-0.463088,0.942139,0.110596


In [8]:
def crossval(drug, train_experiments, test_experiments, train_rna, test_rna):
    train_experiments_now = train_experiments[train_experiments['DRUG']==drug]
    test_experiments_now = test_experiments[test_experiments['DRUG']==drug]

    train_data_and_labels = train_rna.merge(train_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    train_data = train_data_and_labels.iloc[:,1:-7]  
    train_label = train_data_and_labels['auc_per_drug']
    
    test_data_and_labels = test_rna.merge(test_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    cell_line = test_data_and_labels['cell_line']
    #print(test_rna)
    #print(test_experiments_now)
    
    test_data = test_data_and_labels.iloc[:,1:-7]  
    test_label = test_data_and_labels['auc_per_drug']
    
    lasso = Lasso(alpha=0.01)    
    #forest = RandomForestRegressor()
    ridge = Ridge(alpha=0.5)
    EN = ElasticNet(alpha=0.1)
    linear = LinearRegression()
    svr = SVR()
    
    lasso.fit(X = train_data, y = train_label)
    prediction_lasso = lasso.predict(test_data)
    
    ridge.fit(X = train_data, y = train_label)
    prediction_ridge = ridge.predict(test_data)
    
    EN.fit(X = train_data, y = train_label)
    prediction_EN = EN.predict(test_data)
    
    #forest.fit(X = train_data, y = train_label)
    #prediction_forest = forest.predict(test_data)
    
    linear.fit(X = train_data, y = train_label)
    prediction_linear = linear.predict(test_data)
    
    
    svr.fit(X = train_data, y = train_label)
    prediction_svr = svr.predict(test_data)
    
    return pd.DataFrame({'label': test_label, 
                         'lasso': prediction_lasso, 
                         'ridge': prediction_ridge, 
                         'elastic': prediction_EN, 
                         'svr': prediction_svr, 
                         #'forest': prediction_forest,
                         'drugs':drug, 
                         'fold': fold, 
                         'cells': cell_line})


In [9]:
crossval(np.array(drugs)[0], train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna )

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.034e+00, tolerance: 3.002e-02
  model = cd_fast.enet_coordinate_descent(


,label,lasso,ridge,elastic,svr,drugs,fold,cells
355,-0.175753,0.312164,-0.157271,0.421704,0.037869,CYTARABINE,4,ACH-000242
369,1.503422,-0.247953,0.254312,-0.015245,-0.015492,CYTARABINE,4,ACH-000461
195,1.438829,0.153608,-0.341893,-0.181113,-0.085518,CYTARABINE,4,ACH-000792
299,-0.577852,-0.482511,-0.313744,-0.148807,-0.134549,CYTARABINE,4,ACH-000769
333,-0.467138,-1.053873,0.368490,-0.818919,-0.082469,CYTARABINE,4,ACH-000587
...,...,...,...,...,...,...,...,...
187,0.148164,-0.191660,-0.500660,-0.331304,-0.263242,CYTARABINE,4,ACH-000549
262,1.101767,-1.101897,-0.748093,-0.880470,-0.075790,CYTARABINE,4,ACH-000775
272,-0.814482,0.709681,0.477109,0.022574,-0.146998,CYTARABINE,4,ACH-000097
354,-0.106309,0.167491,0.196172,-0.229596,-0.320775,CYTARABINE,4,ACH-000972


In [10]:
#all_tests = [crossval(drug, train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna ) for drug in tqdm(np.array(drugs))]

In [11]:
#results = Parallel(n_jobs=2)(delayed(process)(i) for i in range(10))
all_tests = Parallel(n_jobs=15)(delayed(crossval)(drug,train_experiments, test_experiments, train_rna, test_rna ) for drug in tqdm(np.array(drugs)[:]))
all_tests

  1%|▌                                        | 15/1135 [00:00<00:22, 50.68it/s]/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy vers

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.232e-01, tolerance: 6.282e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.183e-01, tolerance: 6.165e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.210e-01, tolerance: 3.343e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.813e-02, tolerance: 6.406e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converg

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.842e-01, tolerance: 6.178e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.831e-01, tolerance: 6.665e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.034e+00, tolerance: 3.351e-02
  model = cd_fast.enet_coordinate_descent(
  9%|███▋                                    | 105/1135 [01:54<21:57,  1.28s/it]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.396e-01, tolerance: 3.098e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check th

...

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.631e-01, tolerance: 3.756e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.516e-01, tolerance: 3.507e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converg

 96%|█████████████████████████████████████▋ | 1095/1135 [24:41<00:59,  1.48s/it]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.080e+00, tolerance: 3.884e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.527e-01, tolerance: 3.681e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check th

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.363e+00, tolerance: 2.970e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.460e-02, tolerance: 3.312e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.17

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.618e-02, tolerance: 3.028e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.336e-01, tolerance: 3.148e-02
  model = cd_fast.enet_coordinate_descent(
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.59

[        label     lasso     ridge   elastic       svr       drugs  fold   
 355 -0.175753  0.312164 -0.157271  0.421704  0.037869  CYTARABINE     4  \
 369  1.503422 -0.247953  0.254312 -0.015245 -0.015492  CYTARABINE     4   
 195  1.438829  0.153608 -0.341893 -0.181113 -0.085518  CYTARABINE     4   
 299 -0.577852 -0.482511 -0.313744 -0.148807 -0.134549  CYTARABINE     4   
 333 -0.467138 -1.053873  0.368490 -0.818919 -0.082469  CYTARABINE     4   
 ..        ...       ...       ...       ...       ...         ...   ...   
 187  0.148164 -0.191660 -0.500660 -0.331304 -0.263242  CYTARABINE     4   
 262  1.101767 -1.101897 -0.748093 -0.880470 -0.075790  CYTARABINE     4   
 272 -0.814482  0.709681  0.477109  0.022574 -0.146998  CYTARABINE     4   
 354 -0.106309  0.167491  0.196172 -0.229596 -0.320775  CYTARABINE     4   
 362 -1.069110 -2.068215 -0.953997 -1.528410 -0.334864  CYTARABINE     4   
 
           cells  
 355  ACH-000242  
 369  ACH-000461  
 195  ACH-000792  
 299  ACH-

In [12]:
all_tests_combined = pd.concat(all_tests, axis=0)
all_tests_combined

,label,lasso,ridge,elastic,svr,drugs,fold,cells
355,-0.175753,0.312164,-0.157271,0.421704,0.037869,CYTARABINE,4,ACH-000242
369,1.503422,-0.247953,0.254312,-0.015245,-0.015492,CYTARABINE,4,ACH-000461
195,1.438829,0.153608,-0.341893,-0.181113,-0.085518,CYTARABINE,4,ACH-000792
299,-0.577852,-0.482511,-0.313744,-0.148807,-0.134549,CYTARABINE,4,ACH-000769
333,-0.467138,-1.053873,0.368490,-0.818919,-0.082469,CYTARABINE,4,ACH-000587
...,...,...,...,...,...,...,...,...
540265,-0.646440,1.078288,0.242971,0.593048,-0.026631,VE-822,4,ACH-000775
540277,0.908530,0.203079,0.882269,0.742348,0.378473,VE-822,4,ACH-000097
540378,-0.544849,-0.973532,-1.101805,-0.497930,-0.669137,VE-822,4,ACH-000972
540403,-0.234249,0.486750,0.048807,0.113300,0.157231,VE-822,4,ACH-000791


In [13]:
all_tests_combined.to_csv('./results/other_models/lasso'+str(fold)+'.csv')